# O problema da empresa

Nosso cliente é uma empresa de cartão de crédito. Eles nos trouxeram um dataset que inclui dados demográficos e dados financeiros recentes __(últimos seis meses)__ de uma amostra de __30.000 titulares__ de contas. Esses dados estão no nível de conta de crédito; em outras palavras, __há uma linha para cada conta__ (você deve sempre esclarecer qual é a definição de linha, em um dataset). As linhas são rotuladas de acordo com se no mês seguinte ao período de dados histórico de seis meses um proprietário de conta ficou inadimplente, ou seja, não fez o pagamento mínimo.

# Objetivo
Seu objetivo é desenvolver um modelo que preveja se __uma conta ficará inadimplente no próximo mês__, de acordo com dados demográficos e históricos. Posteriormente no livro, discutiremos a aplicação prática do modelo.

## Importando packeges

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import plotly.express as px 
import seaborn as sns 


## Obtendo dados

In [2]:
df = pd.read_excel(r'../data/external/default_of_credit_card_clients__courseware_version_1_21_19.xls')
df.sample(3)

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
6587,c51c1df3-c5be,280000,2,6,3,38,Not available,0,0,-1,...,232,-94,143864,8024,8034,232,94,145935,5000,0
22600,ad305a0e-663d,180000,2,6,2,47,0,0,0,0,...,150812,123957,55778,6028,7758,5188,4570,1876,1701,1
10697,6506edda-c0d5,280000,2,1,1,57,-1,-1,-1,2,...,4795,5328,4969,1669,4795,0,4115,0,0,0


In [3]:
## Verificando quantidade de linhas e colunas
df.shape

(30000, 25)

# Etapas da exploração de dados
1. Saber quantas colunas os dados contêm.

    Podem ser de características, resposta ou metadados.
2. Quantas linhas (amostras).
3. Que tipos de características existem. Quais são categóricas e quais são numéricas?

    Os valores das características categóricas pertencem a classes discretas como “sim”, “não” ou “talvez”. Normalmente as características numéricas pertencem a uma escala numérica contínua, como as quantias em dólares.
4. Qual é a aparência dos dados segundo essas características.

    Para saber isso, você pode examinar o intervalo de valores em características numéricas ou a frequência de classes diferentes em características categóricas, por exemplo.
5. Há dados faltando?

## Verificando a integridade básica dos dados

In [4]:
# nomes das colunas
df.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_1',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default payment next month'],
      dtype='object')

### Dicionário de dados
__LIMIT_BAL:__ Valor do crédito fornecido (em novos dólares taiwaneses (NT)) inclu-
sive o crédito do consumidor individual e familiar (complementar). SEX: Gênero (1 = masculino; 2 = feminino).
__Nota:__ Não usaremos os dados de gênero para tomar decisões de solvibilidade devido a considerações éticas.
__EDUCATION:__ Instrução (1 = pós-graduação; 2 = universidade; 3 = ensino médio; 4 = outros).
__MARRIAGE:__ Estado civil (1 = casado; 2 = solteiro; 3 = outros). AGE: Idade (ano).
__PAY_1–Pay_6:__ Registro de pagamentos passados. Pagamentos mensais passados, registrados de abril a setembro, são armazenados nessas colunas.
__PAY_1__ representa o status de reembolso em setembro; 
__PAY_2__ = status de reem- bolso em agosto; e assim por diante até __PAY_6__, que representa o status de reem- bolso em abril.

A __escala de medida__ do status de reembolso é a seguinte: 
    * -1= pagamento pontual; 
    * 1 = atraso de um mês no pagamento; 
    * 2 = atraso de dois meses no pagamento; 
    * e assim por diante até 8 = atraso de oito meses no pagamento; 
    * 9 = atraso de nove meses ou mais no pagamento.

__BILL_AMT1–BILL_AMT6:__ Valor da fatura (em novos dólares taiwaneses).

__BILL_AMT1__ representa o valor da fatura em setembro; 

__BILL_AMT2__ representa o valor da fatura em agosto; e assim por diante até:

__BILL_AMT7__, que representa o valor da fatura em abril.

__PAY_AMT1–PAY_AMT6__: Valor de pagamentos anteriores (novos dólares taiwaneses). 

__PAY_AMT1__ representa o valor pago em setembro; 
__PAY_AMT2__ representa o valor pago em agosto; e assim por diante até __PAY_AMT6__
, que representa o valor pago em abril.

In [5]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,798fc410-45c1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,8a8c8f3b-8eb4,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,85698822-43f5,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,0737c11b-be42,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,3b7f77cc-dbc0,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [6]:
df['ID'].nunique()

29687

In [7]:
df['ID'].value_counts()

ad23fe5c-7b09    2
1fb3e3e6-a68d    2
89f8f447-fca8    2
7c9b7473-cc2f    2
90330d02-82d9    2
                ..
2edeb3a6-d38d    1
27e11893-92e9    1
2b1c282b-441e    1
bd737997-0eb0    1
15d69f9f-5ad3    1
Name: ID, Length: 29687, dtype: int64

In [8]:
id_counts = df['ID'].value_counts()
id_counts.head()

ad23fe5c-7b09    2
1fb3e3e6-a68d    2
89f8f447-fca8    2
7c9b7473-cc2f    2
90330d02-82d9    2
Name: ID, dtype: int64

In [9]:
# Exiba o número de entradas duplicadas agrupadas executando outra conta- gem de valores:
id_counts.value_counts()

1    29374
2      313
Name: ID, dtype: int64

Na saída anterior e na contagem de valores inicial, podemos ver que a maioria dos IDs ocorre exatamente uma única vez, como esperado. No entanto, __313 IDs ocorrem duas vezes__. *Logo, nenhum ID ocorre mais do que duas vezes*. De posse dessas informações, estamos prontos para começar a examinar mais detalhadamente esse problema de qualidade de dados e corrigi-lo. Criaremos máscaras booleanas para limpar melhor os dados.

## Máscaras booleanas



In [10]:
import numpy as np

In [11]:
np.random.seed(24)

random_intergers = np.random.randint(low=1,high=5, size=100)

random_intergers

array([3, 4, 1, 4, 2, 2, 2, 1, 4, 4, 1, 4, 4, 3, 4, 4, 4, 4, 4, 4, 2, 3,
       4, 4, 2, 4, 2, 4, 1, 1, 3, 1, 4, 2, 2, 1, 4, 3, 2, 2, 3, 3, 2, 3,
       4, 2, 3, 4, 1, 3, 4, 2, 2, 4, 1, 2, 3, 3, 2, 1, 2, 4, 2, 1, 4, 3,
       1, 2, 1, 3, 4, 2, 3, 4, 1, 3, 2, 1, 1, 3, 3, 4, 3, 2, 4, 1, 4, 4,
       4, 4, 1, 4, 3, 2, 4, 4, 3, 3, 2, 4])

In [12]:
is_igual_to_3 = random_intergers == 3

In [13]:
is_igual_to_3

array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False,  True, False, False,  True,  True, False,  True, False,
       False,  True, False, False,  True, False, False, False, False,
       False, False,  True,  True, False, False, False, False, False,
       False, False,  True, False, False, False,  True, False, False,
        True, False, False,  True, False, False, False,  True,  True,
       False,  True, False, False, False, False, False, False, False,
       False, False,  True, False, False, False,  True,  True, False,
       False])

In [14]:
is_igual_to_3[:5]

array([ True, False, False, False, False])

In [15]:
len(random_intergers[is_igual_to_3])

22

## Continuando a verificação da integridade dos dados

In [16]:
id_counts

ad23fe5c-7b09    2
1fb3e3e6-a68d    2
89f8f447-fca8    2
7c9b7473-cc2f    2
90330d02-82d9    2
                ..
2edeb3a6-d38d    1
27e11893-92e9    1
2b1c282b-441e    1
bd737997-0eb0    1
15d69f9f-5ad3    1
Name: ID, Length: 29687, dtype: int64

In [17]:
dupe_mark = id_counts == 2
dupe_mark[0:5]

ad23fe5c-7b09    True
1fb3e3e6-a68d    True
89f8f447-fca8    True
7c9b7473-cc2f    True
90330d02-82d9    True
Name: ID, dtype: bool

In [18]:
dupe_ids=id_counts.index[dupe_mark]
len(dupe_ids)

313

In [19]:
dupe_ids[:5]

Index(['ad23fe5c-7b09', '1fb3e3e6-a68d', '89f8f447-fca8', '7c9b7473-cc2f',
       '90330d02-82d9'],
      dtype='object')

In [20]:
df.loc[df['ID'].isin(dupe_ids[0:3]),:].head(10)

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
5033,89f8f447-fca8,320000,2,2,1,32,0,0,0,0,...,169371,172868,150827,8000,8000,5500,6100,6000,5000,0
5133,89f8f447-fca8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16727,1fb3e3e6-a68d,80000,1,2,2,33,2,2,0,0,...,27394,29922,31879,0,2000,2000,3000,2600,0,1
16827,1fb3e3e6-a68d,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29685,ad23fe5c-7b09,50000,1,3,1,32,0,0,0,0,...,12882,8131,3983,3000,2871,1000,163,3983,3771,1
29785,ad23fe5c-7b09,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


O que estamos vendo aqui é que cada ID duplicado parece ter uma linha de dados válidos e outra somente com zeros. Pare um momento e pense o que poderia fazer com essa informação

In [21]:
df_zero_mask = df == 0
df_zero_mask

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,False,True,True,True,True,False
1,False,False,False,False,False,False,False,False,True,True,...,False,False,False,True,False,False,False,True,False,False
2,False,False,False,False,False,False,True,True,True,True,...,False,False,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,True,True,True,True,...,False,False,False,False,False,False,False,False,False,True
4,False,False,False,False,False,False,False,True,False,True,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,False,False,False,False,False,False,True,True,True,True,...,False,False,False,False,False,False,False,False,False,True
29996,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,True,True,True
29997,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,True,False,False,False,False,False
29998,False,False,False,False,False,False,False,False,True,True,...,False,False,False,False,False,False,False,False,False,False


In [22]:
feature_zero_mask = df_zero_mask.iloc[:,1:].all(axis=1)
sum(feature_zero_mask)

315

In [25]:
# Crie uma matriz booleana
df_zero_mask = df == 0

In [28]:
# Crie a série booleana feature_zero_mask,
feature_zero_mask = df_zero_mask.iloc[:,1:].all(axis=1)
feature_zero_mask

0        False
1        False
2        False
3        False
4        False
         ...  
29995    False
29996    False
29997    False
29998    False
29999    False
Length: 30000, dtype: bool

In [29]:
# Calcule a soma da série booleana usando este comando:
sum(feature_zero_mask)

315

In [30]:
# limpe o DataFrame eliminado as linhas só com zeros
df_clean_1 = df.loc[~feature_zero_mask,:].copy()

In [32]:
df_clean_1.shape

(29685, 25)

In [35]:
#  Obtenha o número de IDs exclusivos executando este código:
df_clean_1['ID'].nunique()

29685

##  Explorando e limpando os dados

In [36]:
df_clean_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29685 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   ID                          29685 non-null  object
 1   LIMIT_BAL                   29685 non-null  int64 
 2   SEX                         29685 non-null  int64 
 3   EDUCATION                   29685 non-null  int64 
 4   MARRIAGE                    29685 non-null  int64 
 5   AGE                         29685 non-null  int64 
 6   PAY_1                       29685 non-null  object
 7   PAY_2                       29685 non-null  int64 
 8   PAY_3                       29685 non-null  int64 
 9   PAY_4                       29685 non-null  int64 
 10  PAY_5                       29685 non-null  int64 
 11  PAY_6                       29685 non-null  int64 
 12  BILL_AMT1                   29685 non-null  int64 
 13  BILL_AMT2                   29685 non-null  in